# RAPIDS: Python-style Data Science on NVIDIA GPU

RAPIDS requires Pascal (6.0) or higher CUDA compute capability

What is CUDA compute capability? https://en.wikipedia.org/wiki/CUDA

There's a grid on that page listing all of the details on models ... but effectively it means a GPU in the most recent generations of either gaming (GeForce), workstation (Quadro), server (Tesla), or embedded (Jetson/Tegra/DRIVE).

We also need CUDA 9.2 or higher. How can we tell what we have?

In [ ]:
! nvidia-smi

# cuDF and cuML Smoke Test

Let's make sure everything in installed and running correctly. (Installation instructions and docs are on the RAPIDS project page at https://rapids.ai)

In [ ]:
import cudf

gdf = cudf.DataFrame({'test':[1,2,3]})
gdf

In [ ]:
import cuml

df_float = cudf.DataFrame()
df_float['0'] = [1.0, 2.0, 5.0]
df_float['1'] = [4.0, 2.0, 1.0]

dbscan_float = cuml.DBSCAN(eps=1.0, min_samples=1)
dbscan_float.fit(df_float)

dbscan_float.labels_

In [ ]:
import cugraph

G = cugraph.Graph()
edges = cudf.DataFrame()
edges['a'] = cudf.Series([0, 1, 2], dtype='int32')
edges['b'] = cudf.Series([1, 2, 0], dtype='int32')
G.from_cudf_edgelist(edges, source='a', destination='b')
G.view_edge_list()

# Add GPU to Pandas-Style Analytics with cuDF

## Beer Review Data Analysis

Now that RAPIDS is maturing, it doesn't make as much sense to demo a handful of API features, or a use case made just to demo RAPIDS. The promise of RAPIDS is easily moving "regular" Python data science workflows to GPU, so today we are going to do exactly that. 

In this lab, I took on of my real Pandas workshop labs, and a dataset used in a recent popular O'Reilly conference, and challenged myself to run it unmodified -- as much as possible -- with RAPIDS.

As we'll see, about 98% of the code is unchanged. If you know Pandas (or learn it) you'll know RAPIDS.

Where I did run into APIs that haven't been implmented yet, instead of hiding that with a slick solution and letting you learn the hard way, I let them "error out" here so we could see together. And then I offered a workaround.

Let's go!

In [ ]:
import cudf

df = cudf.read_csv('data/beer_small.csv')

df

How many reviews are there?

In [ ]:
len(df)

How can we tell if there are missing values?

In [ ]:
df.count()

Since most reviews have data for most fields, let's drop the records with incomplete data

In [ ]:
df2 = df.dropna()

In [ ]:
df2.count()

Let's get summary statistics for the numeric columns ... things like review score and ABV

In [ ]:
df2.describe()

There are some really low-alcohol beers in there ... maybe even bogus data.

Find all entries with ABV less than 1%

In [ ]:
low_abv = df2[df2.beer_abv < 1]

low_abv

How many of these reviews are there?

In [ ]:
len(low_abv)

This includes multiple reviews for the same beer, so let's group by beer and count.

In [ ]:
grouping = low_abv.groupby('beer_name')
grouping.size()

How consistent are the O'Douls overall scores?

In [ ]:
scores = low_abv[low_abv.beer_name=="O'Doul's"]['review_overall']
scores

Let's plot a histogram

In [ ]:
try:
    scores.hist()
except Exception as err:
    print(err)

__Ok, we've run into our first Pandas incompatibility__

Happily, we can always convert `to_pandas()` if needed. Whether that makes sense depends on the task and the size of the data. Here we want to plot, so we can convert or -- if we are interested in the approximate distribution but not exact numbers -- we can downsample and plot. (Example of downsampling comes later.)

In [ ]:
%matplotlib inline

scores.to_pandas().hist()

__Note:__ As RAPIDS evolves, it adds more and more Pandas compatibility, so definitely check back in the future and don't assume that if a feature is missing today that it won't arrive soon.

---

What are the mean and sd for the O'Doul's overall scores?

In [ ]:
scores.mean(), scores.std()

In the full dataset, can we count reviews by brewery, and then by style within that brewery?

In [ ]:
df2.groupby(['brewery_name', 'beer_style']).size()

### Now we'll try and build up a slightly more complex report

Step 1: Find all rows corresponsing to reviews where the beer style starts with "American"

In [ ]:
all_american = df2[df2.beer_style.str.startswith('American')]
all_american

Next, make a dataframe with just the `beer_style` and `review_overall` fields for those rows.

In [ ]:
narrowed = all_american[['beer_style', 'review_overall']]
narrowed

Now we'll make a boxplot to capture the range and variance of the ratings. Pandas will do all the work if we call the built-in API. Look for it here: https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html

In [ ]:
narrowed.to_pandas().boxplot(by='beer_style', vert=False, figsize=(12,10))

# Add GPU to Scikit-Learn-Style Modeling with cuML

As with the cuDF example, we'll start with a real Pandas + scikit-learn modeling exercise, and see if we can move it to RAPIDS with minimal effort.

And, as before, if anything that didn't port over directly, I've allowed it to fail, and then offered a workaround.

## Dataset: Diamonds

This dataset of diamond sales (http://ggplot2.tidyverse.org/reference/diamonds.html) is of moderate size (~55,000 records) and resembles data records that occur in many business scenarios.

For each of the diamond sales records, we have the following properties:
* price: price in US dollars (\\$326-\\$18,823)
* carat: weight of the diamond (0.2-5.01)
* cut: quality of the cut (Fair, Good, Very Good, Premium, Ideal)
* color: diamond colour, from J (worst) to D (best)
* clarity: a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
* x: length in mm (0-10.74)
* y: width in mm (0-58.9)
* z: depth in mm (0-31.8)
* depth: total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43-79)
* table: width of top of diamond relative to widest point (43-95)

In [ ]:
import cudf 

df = cudf.read_csv('data/diamonds.csv')

df.head(5)

The "unnamed" column is a row number in the dataset. It turns out that this row number -- which sounds like it should be meaningless -- actually leaks key data about the diamonds. Why? It looks like that dataset is composed of merging 3 or more sets of records, was never shuffled, and some of those subsets were in price-ascending order.

In [ ]:
df['price']

To demo the "information leakage by row" phenomenon, let's downsample the data (to make it easier to plot) and then graph row number against price.

In [ ]:
try:
    smaller_df = df.sample(frac=0.05)
except Exception as err:
    print(err)

__Doh!__ Looks like we've hit another incompatibility the current release. That's ok, we can easily downsample by using another tool to generate random numbers, and then do some filtering.

This also gives us a chance to see a RAPIDS highlight: on-GPU ("on-device") interop with datasets in other popular GPU accelerated tools, including PyTorch and CuPy.

In [ ]:
import cupy

random_vals = cupy.random.rand(len(df))

cudf.Series(random_vals)

In [ ]:
mask = cudf.Series(random_vals)

In [ ]:
sample = df[mask < 0.0002]
sample

In [ ]:
sample.columns

In [ ]:
records = df[['Unnamed: 0', 'price']][mask < 0.05].to_pandas().astype('float32')

In [ ]:
records.plot.scatter('Unnamed: 0', 'price', s=0.05, figsize=(10,8))

__OK, that worked!__ But we have a GPU ... surely that's the perfect device for high-volume rendering! 

We could use tools like Graphistry (part of RAPIDS: https://www.graphistry.com/ and https://github.com/graphistry/pygraphistry) to do that.

---

Let's get rid of the row number:

In [ ]:
df2 = df.drop(df.columns[0], axis=1)

df2[:3]

__Categorical Feautres__

Now ... computers are good with numbers, but what about those words? ("Premium", "Ideal", etc.) It turns out that not only do we need to convert them to numbers, but we often want to do that in a way that treats them as totally separate properties.

We'll consider the "Ideal"-ness of a diamond totally separately from the "Premium"-ness of that diamond, etc., and of course each diamond only has one of those properties. This is called "one-hot encoding" (or sometimes "dummy variable encoding" or "one of k encoding") and not always the right solution, but it's a common one.

In [ ]:
df2.cut.value_counts()

In [ ]:
df2.cut.value_counts().to_pandas().plot(kind='bar')

In [ ]:
df2.dtypes

In [ ]:
df3 = cudf.get_dummies(df2)

df3.iloc[:3, 7:18]

Now we'll split out a "test set" -- remember we want to be able to evaluate the model on records that it hasn't seen before.

In [ ]:
from cuml.preprocessing.model_selection import train_test_split

y = df3.price.to_frame() ### minor API DIFFERENCE alert

df3.drop_column('price') ## minor API DIFFERENCE alert

X = df3

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7) ## minor API DIFFERENCE

#### Baselines

In this case we'll use the mean price of the diamonds as a (constant) baseline model:

In [ ]:
y_train.mean()

So our first "baseline" model just says for any diamond we might look at, its price is about $3900. Obviously this is usually going to be wrong, and often by a lot. But it's better than nother. Later we'll see how to compare a "real" model against this one.

Next, we'll set up a model. Let's start with something simple like kNN (k-nearest-neighbors)

__`scikit-learn` code:__

```python
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor(n_neighbors=5)
model = neigh.fit(X_train, y_train) 
```

And the RAPIDS code is a touch different, because it matches a more general `scikit` API, `NearestNeighbors` (https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors) and we'll do the regression part ourselves.

In [ ]:
from cuml.neighbors import KNeighborsRegressor

nn = KNeighborsRegressor(n_neighbors=3)
nn.fit(X_train, y_train)

The inference (prediction) part

In [ ]:
y_pred = nn.predict(X_test)

Ok, how did we do?

For regression problems like this, we'll measure the accuracy of our predictions using RMSE (root mean squared error). This is a measure of "how wrong" we typically are in our predictions, measured in the units we are predicting (i.e., in this case, dollars).

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
import numpy as np

np.sqrt(((y_test.reset_index().price - y_pred[0])**2).mean())

So is that actually any good?

One way to get an idea is to compare it to the standard deviation of the data:

In [ ]:
y_test.std()

__Not bad__ ... it's pretty much what we "should" get for 3 nearest neighbors in this dataset.

## Parametric Model: Linear Regression

The canonical example of a parametric model is a linear regression model. Linear regression -- which you might have done by hand on a small amount of data in high school or a college stats class -- is simple, fast, robust, and performs reasonably well for many kinds of real-world data.

In fact, linear regression is one of the two or three most widely used algorithms in the world for data modeling.

Here's a simple version with one predictor and one response plotted against each other, along with a regression line:

<img src="images/gyP3KGA.png">

Since we're not doing a stats/ML class here, we'll skip the details on how we can fit a linear regression mathematically, and focus on comparing `scikit-learn` to RAPIDS for the task:

Scikit code:
```python
from sklearn import linear_model

lr = linear_model.LinearRegression()
linear = lr.fit(X_train, y_train)

y_pred = linear.predict(X_test)
print("RMSE %f" % np.sqrt(mean_squared_error(y_test, y_pred)) )
```

RAPIDS code:

In [ ]:
from cuml import linear_model

lr = linear_model.LinearRegression()

try:
    reg = lr.fit(X_train, y_train)
except Exception as err:
    print(err)

Ok, it wasn't quite that simple ... but it's a good idea to have uniform data types for performance purposes.
We can write a helper to set all the column data widths.

*Be careful not to mix precisions (widths). If you're lucky, you'll get an error; if not, you may get bad output.*

In [ ]:
def to_float(df):
    for col in df.columns:
        new_col = df[col].astype('float64')
        df.drop_column(col)
        df[col] = new_col
        return df
    
to_float(X_train)

And now we'll train

In [ ]:
reg = lr.fit(X_train, to_float(y_train).price)

print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)

How did we do? Let's predict for the test set.

In [ ]:
preds = lr.predict(to_float(X_test))

print("Predictions:")
print(preds)

In [ ]:
np.sqrt(((y_test.reset_index().price - preds)**2).mean())

That's exactly where we expect to end up for this data and model type.

## Lab: Powerplant Output 

https://archive.ics.uci.edu/ml/datasets/combined+cycle+power+plant

About the business problem: peaker plant operation

What is in this dataset? Just under 10,000 observations of:

* Temperature (AT) in the range 1.81°C and 37.11°C
* Ambient Pressure (AP) in the range 992.89-1033.30 millibar,
* Relative Humidity (RH) in the range 25.56% to 100.16%
* Exhaust Vacuum (V) in the range 25.36-81.56 cm Hg
* Net hourly electrical energy output (PE) 420.26-495.76 MW

What is the goal? To model output (PE) based other measurements

In [ ]:
df = cudf.read_csv('data/powerplant.csv')

df

Try to build a linear regression model for power output. (Hint: you can cut/paste a lot of the code we've already used in this notebook!)